In [15]:
import os
import glob
import h5py

import numpy as np

from skimage import io, color, exposure, transform
from sklearn.model_selection import train_test_split

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras import backend
backend.set_image_data_format('channels_first')

from matplotlib import pyplot
%matplotlib inline

In [14]:
NUM_CLASSES = 43
IMG_SIZE = 48

<h2> Preprocessing </h2>

1. Histogram normalization in HSV yellow
2. Crop central region
3. Resize
4. Roll RGB axis to 0

In [6]:
def preprocess(image):
    
    # Histogram normalization in HSV yellow
    temp = color.rgb2hsv(image)
    temp[:,:,2] = exposure.equalize_hist(temp[:,:,2])
    image = color.hsv2rgb(temp)
    
    # Crop central region
    ms = min(image.shape[:-1])
    center = image.shape[0]//2, image.shape[1]//2
    image = image[
        center[0] - ms//2 : center[0] + ms//2,
        center[1] - ms//2 : center[1] + ms//2,
        :
    ]
    
    # Resize
    image = transform.resize(image, (IMG_SIZE, IMG_SIZE))
    
    # Roll RGB axis to 0
    
    